In [1]:
import pandas as pd

# 1. 读取并合并数据，纵向连接
df_2015 = pd.read_csv(r'C:\Users\FeatherWounds\Downloads\2015年国内主要城市年度数据.csv')
df_2016 = pd.read_csv(r'C:\Users\FeatherWounds\Downloads\2016年国内主要城市年度数据.csv')
df_2017 = pd.read_csv(r'C:\Users\FeatherWounds\Downloads\2017年国内主要城市年度数据.csv')

# 合并三个数据框
merged_df = pd.concat([df_2015, df_2016, df_2017], axis=0)

In [2]:
# 2. 按照年份来聚合
grouped_by_year = merged_df.groupby('年份')

In [3]:
# 3. 求每年的国内生产总值
gdp_by_year = grouped_by_year['国内生产总值'].sum()
print("每年的国内生产总值:")
print(gdp_by_year)

每年的国内生产总值:
年份
2015    277958.37
2016    300696.10
2017    332562.43
Name: 国内生产总值, dtype: float64


In [4]:
# 4. 处理缺省值，填充为0
filled_df = merged_df.fillna(0)
print("\n处理缺省值后的数据框前几行:")
print(filled_df.head())


处理缺省值后的数据框前几行:
     地区    年份    国内生产总值  第一产业增加值  第二产业增加值   第三产业增加值  社会商品零售总额    货物进出口总额  \
0    北京  2015  23014.59   140.21  4542.64  18331.74   10338.0  319416.16   
1    天津  2015  16538.19   208.82  7704.22   8625.15    5257.3  114282.80   
2   石家庄  2015   5440.60   494.44  2452.40   2493.77    2693.0   12160.29   
3    太原  2015   2735.34    37.40  1020.18   1677.77    1540.8   10677.38   
4  呼和浩特  2015   3090.52   126.23   867.08   2097.21    1353.5    2072.75   

     年末总人口  在岗职工平均工资  普通高等学校在校学生数  医院、卫生院数  房地产开发投资额  
0  1345.20    113073        60.36      701   4177.05  
1  1026.90     81486        51.29      661   1871.55  
2  1028.84     54441        41.98      393    965.13  
3   367.39     60516        42.14      247    597.83  
4   238.58     53698        23.52      174    509.05  


In [6]:
# 5. 计算每个城市2015-2017年GDP的年均增长率，并找出增长率最高和最低的五个城市
# 计算年均增长率
def calculate_growth(df):
    gdp_2015 = df[df['年份'] == 2015].set_index('地区')['国内生产总值']
    gdp_2017 = df[df['年份'] == 2017].set_index('地区')['国内生产总值']
    growth_rate = ((gdp_2017 / gdp_2015) ** (1/2) - 1) * 100  # 年均增长率
    return growth_rate

growth_rates = calculate_growth(merged_df).sort_values(ascending=False)

# 找出增长率最高和最低的五个城市
top5 = growth_rates.head(5)
bottom5 = growth_rates.tail(5)

print("5. GDP年均增长率最高和最低的五个城市:")
print("\n增长率最高的五个城市:")
print(top5)
print("\n增长率最低的五个城市:")
print(bottom5)

5. GDP年均增长率最高和最低的五个城市:

增长率最高的五个城市:
地区
西安    13.474172
成都    13.398259
深圳    13.355030
合肥    12.889341
拉萨    12.788795
Name: 国内生产总值, dtype: float64

增长率最低的五个城市:
地区
哈尔滨      5.118674
乌鲁木齐     2.109128
大连      -2.406989
呼和浩特    -5.777610
沈阳     -10.195788
Name: 国内生产总值, dtype: float64


In [7]:
# 6. 对医院、卫生院数进行归一化处理（Min-Max标准化），并按年份比较各城市医疗资源的变化
# Min-Max标准化
def min_max_normalize(df, column):
    grouped = df.groupby('年份')[column]
    min_val = grouped.transform('min')
    max_val = grouped.transform('max')
    df[f'normalized_{column}'] = (df[column] - min_val) / (max_val - min_val)
    return df

normalized_df = min_max_normalize(merged_df, '医院、卫生院数')

# 按年份比较医疗资源变化
medical_resource_changes = normalized_df.pivot_table(index='地区', 
                                                   columns='年份', 
                                                   values='normalized_医院、卫生院数')

print("\n6. 各城市医疗资源归一化值按年份比较:")
print(medical_resource_changes.head())


6. 各城市医疗资源归一化值按年份比较:
年份        2015      2016      2017
地区                                
上海    0.388079  0.385511  0.389535
乌鲁木齐  0.077483  0.059508  0.115116
兰州    0.070199  0.072445  0.115116
北京    0.425828  0.422380  0.730233
南京    0.102649  0.106727  0.223256


In [8]:
# 7. 提取北京、上海、广州、深圳四个城市2015-2017的GDP和社会商品零售总额数据
target_cities = ['北京', '上海', '广州', '深圳']
selected_data = merged_df[merged_df['地区'].isin(target_cities)][
    ['地区', '年份', '国内生产总值', '社会商品零售总额']]

# 保存为新的CSV文件
selected_data.to_csv('四大城市GDP和消费数据.csv', index=False, encoding='utf-8-sig')

print("\n7. 已提取四大城市数据并保存为'四大城市GDP和消费数据.csv'")


7. 已提取四大城市数据并保存为'四大城市GDP和消费数据.csv'
